 <b><p style="font-size:350%;  color:#363636"> IBM Course 8 Final Project  Part2 </p></b>

In [1]:
# import necessary libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline
import datetime as dt

import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [4]:


# Load the data using pandas
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv")
data.rename(columns={'unemployment_rate':'Unemployment_Rate'}, inplace=True)
# Initialize the Dash app
app = dash.Dash(__name__)

# Create the list of years 
year_list = [i for i in range(1980, 2024)]

    # TASK 2.1: Create a Dash application
app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",  
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}),
    
    # TASK 2.2: Add drop-down menus
    html.Div([
        html.Label("Select Statistics:"), 
        dcc.Dropdown(
            id='dropdown-statistics',
            options=[
                {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
            ],
            placeholder='Select a report type',
            value='Yearly Statistics',
            style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'text-align-last': 'center'}
        ),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(i), 'value': i} for i in year_list],
            placeholder='Select a year',
            style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'textAlignLast': 'center'}
        ),
        # TASK 2.3: Add a division for output display
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

#TASK 2.4: Creating Callbacks; Define the callback function
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    [Input(component_id='dropdown-statistics', component_property='value')]
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True


@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_statistics, input_year):

    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

         
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                           labels={'Automobile_Sales': 'Average Automobile Sales'},
                           title="Automobile Sales Fluctuation Over Recession Period"))

        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales',
                          labels={'Automobile_Sales': 'Average Automobile Sales', 'Vehicle_Type': 'Vehicle Type'},
                          title="Average Vehicles Sold by Vehicle Type During Recession"))

        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                          title="Expenditure Share by Vehicle Type During Recession"))

        unemployment_data = recession_data.groupby('Vehicle_Type')['Unemployment_Rate'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemployment_data, x='Vehicle_Type', y='Unemployment_Rate',
                          labels={'Unemployment_Rate': 'Unemployment Rate', 'Vehicle_Type': 'Vehicle Type'}, 
                          title="Effect of Unemployment Rate on Vehicle Type and Sales"))

         # Return a Div containing all four charts arranged in a flexible layout
        return html.Div([
            html.Div(children=[R_chart1], style={'width': '50%'}),
            html.Div(children=[R_chart2], style={'width': '50%'}),
            html.Div(children=[R_chart3], style={'width': '50%'}),
            html.Div(children=[R_chart4], style={'width': '50%'})
        ], style={'display': 'flex', 'flex-wrap': 'wrap', 'justify-content': 'space-around'})

    
    elif selected_statistics == 'Yearly Statistics' and input_year:
        yearly_data = data[data['Year'] == int(input_year)]

                                      
#plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales',
                          labels={'Automobile_Sales': 'Average Automobile Sales'}, 
                          title="Yearly average automobile sales"))
            
# Plot 2 Total Monthly Automobile sales using line chart.
        mas = data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(mas, x= 'Month', y='Automobile_Sales',
                            labels={'Automobile_Sales': ' Automobile Sales'},
                            title= 'Total Monthly Automobile Sales'))

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby(['Year', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph( figure =px.bar(avr_vdata,x='Vehicle_Type', y='Automobile_Sales',
                            labels={'Automobile_Sales': 'Average Automobile Sales', 'Vehicle_Type': 'Vehicle Type'},
                            title='Average vehicles sales by type in the {}'.format(input_year)))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby(['Year', 'Vehicle_Type'])['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values = 'Advertising_Expenditure', 
                                title='Percentage of ad-expenditure by vehicle type in {}'.format(input_year),
                                 names='Vehicle_Type'))


        
        #TASK 2.6: Returning the graphs for displaying Yearly data
        return html.Div([
            html.Div(children=[Y_chart1], style={'width': '50%'}),
            html.Div(children=[Y_chart2], style={'width': '50%'}),
            html.Div(children=[Y_chart3], style={'width': '50%'}),
            html.Div(children=[Y_chart4], style={'width': '50%'})
        ], style={'display': 'flex', 'flex-wrap': 'wrap', 'justify-content': 'space-around'})

if __name__ == '__main__':
    app.run_server(debug=True)

